In [4]:
from bs4 import BeautifulSoup
import re
import requests
import fake_useragent
import json
import csv
import random
import time
import os

ua = fake_useragent.UserAgent( )

headers = {
    "Accept": "*/*",
    "User-Agent": ua.ie,
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
}

In [5]:
dic = {'А':'A', 'а':'a', 'В':'B', 'Е':'E', 'е':'e', 'К':'K', 'к':'k', 'М':'M', 'Н':'H', 'О':'O', 'о':'o', 'Р':'P', 'р':'p', 
       'С':'C', 'с':'c', 'Т':'T', 'У':'Y', 'у':'y', 'Х':'X', 'х':'x'}

def Rustoeng(str):
    res=''
    for i in range(0,len(str)):
        if str[i] in dic:
            res =  res + dic[str[i]]
        else:
            res = res + str[i]
        
    #   res = dic.get(st[i], st[i])
    return res


In [6]:
url = 'https://teko-com.ru/katalog/'
req = requests.get(url, headers=headers)
    
soup = BeautifulSoup(req.content, 'lxml')
all_sensors_href = soup.find('ul', class_="catalog-prod-ul").find_all('a')
all_catagories_dict = {}

for item in all_sensors_href:
    item_text = item.text
    item_href = 'https://teko-com.ru' + item.get('href')
    
    all_catagories_dict[item_text] = item_href
    
with open ('all_catagories_dict.json', "w") as file:
    json.dump(all_catagories_dict, file, indent = 4, ensure_ascii=False)


In [7]:
with open ('all_catagories_dict.json') as file:
    all_catagories = json.load(file)

print (all_catagories)

{'Индуктивные датчики': 'https://teko-com.ru/katalog/induktivnye-datchiki/', 'Оптические датчики': 'https://teko-com.ru/katalog/opticheskie-datchiki/', 'Емкостные датчики': 'https://teko-com.ru/katalog/emkostnye-datchiki/', 'Сенсорные кнопки': 'https://teko-com.ru/katalog/sensornye-knopki/', 'Магниточувствительные датчики': 'https://teko-com.ru/katalog/magnitochuvstvitelnye-datchiki/', 'Датчики уровня': 'https://teko-com.ru/katalog/datchiki-urovnja/', 'Ультразвуковые датчики': 'https://teko-com.ru/katalog/ultrazvukovie-datchiki/', 'Приборы и средства автоматизации': 'https://teko-com.ru/katalog/pribory/', 'Взрывозащищенное оборудование': 'https://teko-com.ru/katalog/vzryvobezopasnoe-oborudovanie/', 'Аксессуары к датчикам': 'https://teko-com.ru/katalog/aksessuary/', 'Конвейерная автоматика': 'https://teko-com.ru/katalog/konvejernaja-avtomatika/', 'Датчики относительной влажности и температуры воздуха': 'https://teko-com.ru/katalog/datchiki-otnositelnoj-vlazhnosti-i-temperatury/', 'Датчи

In [8]:
iteration_count = int(len(all_catagories))-1
count = 0
print (f'Всего итераций: {iteration_count}')
for category_name, category_href in all_catagories:
    
    
    count +=1
    print (f'# Итерация {count}. {category_name}')
    iteration_count = iteration_count-1
    time.sleep(random.randrange(2,6))


Всего итераций: 15


ValueError: too many values to unpack (expected 2)

In [9]:
def back_up():
    cur_time = time.localtime(time.time())
    os.rename("data",f"back_up_{cur_time.tm_mday}_{cur_time.tm_mon}_{cur_time.tm_year}")

# with open(f'data/{category_name}.csv', 'w', encoding = 'utf-8', newline='') as file:
    # writer = csv.writer(file)
    # writer.writerow(('category_name', 'sensor_type', 'sensor_symbol', 'sensor_url', 'sensor_price', 'sensor_in_stock'))

In [10]:
category_name = 'Индуктивные датчики'
category_href = 'https://teko-com.ru/katalog/induktivnye-datchiki/'
    
pages = 6
for i in range(1,pages):
    
    url_page = category_href + '/?page='+str(i)
    res = requests.get(url_page, headers=headers)
    soup = BeautifulSoup(res.content, 'lxml')
    sensor_list = soup.find_all('div', class_="caption")
    if len(sensor_list) == 0:
        break
    for sensor in sensor_list:
        sensor_bare_name = sensor.find('div', class_="tovarszg").text
        sensor_name = re.sub ( '\(.+\)', '', sensor_bare_name).strip()
        sensor_type = (re.sub("[a-zA-Z0-9-,]+", '', sensor_name).strip())
        sensor_type = ' '.join(filter(lambda x: len(x) > 3, sensor_type.split()))      #удалить короткие слова, фильтр русских букв в названии датчика попавших из обозначения
        sensor_symbol = Rustoeng(sensor_name.replace(sensor_type, '').strip())          #Транслитерация русских букв в обозначении
        sensor_url = sensor.find('a').get("href")
        sensor_price = sensor.find('span', class_="category-price-test").text.strip()
        sensor_in_stock = sensor.find('p', class_="pricers").find('span').text.strip()
        print (f'{category_name} - {sensor_type} - {sensor_symbol}: {sensor_url} - {sensor_price} - {sensor_in_stock}')
        with open(f'data/{category_name}.csv', 'a', encoding = 'utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow((category_name, sensor_type, sensor_symbol, sensor_url, sensor_price, sensor_in_stock))
    
    time.sleep(random.randrange(2,6))

Индуктивные датчики - Бесконтактный выключатель - ISB AF4B-31P-5-LZ-4,5: https://teko-com.ru/index.php?route=product/product&path=1437&product_id=17324 - 1 350 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISN ET8A-31N-15-LZ: https://teko-com.ru/product/isn-et8a-31n-15-lz.html - 1 990 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISB A4B-31P-5-LZ-C: https://teko-com.ru/product/isb-a4b-31p-5-lz-c.html - 2 050 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISB A4A8-31P-5F-LZT1-C-P: https://teko-com.ru/product/isb-a4a8-31p-5f-lzt1-c-p.html - 1 740 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISN F4A-31P-8-LZ: https://teko-com.ru/product/isn-f4a-31p-8-lz.html - 1 490 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISB AC12S-31P-1,5-LS4-H: https://teko-com.ru/product/isb-ac12s-31p-1-5-ls4-h.html - 3 780 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISB B1B-31P-1,5F-LP-C: https://teko-com.ru/p

In [11]:
soup = BeautifulSoup(res.content, 'lxml')
sensor_list = soup.find_all('div', class_="caption")
for sensor in sensor_list:
    
    
    sensor_bare_name = sensor.find('div', class_="tovarszg").text
    sensor_name = re.sub ( '\(.+\)', '', sensor_bare_name).strip()
    sensor_type = (re.sub("[a-zA-Z0-9-,]+", '', sensor_name).strip())
    sensor_type = ' '.join(filter(lambda x: len(x) > 3, sensor_type.split()))      #удалить короткие слова, фильтр русских букв в названии датчика.
    sensor_symbol = sensor_name.replace(sensor_type, '').strip()
    sensor_url = sensor.find('a').get("href")
    sensor_price = sensor.find('span', class_="category-price-test").text.strip()
    sensor_in_stock = sensor.find('p', class_="pricers").find('span').text.strip()
    print (f'{category_name} - {sensor_type} - {sensor_symbol}: {sensor_url} - {sensor_price} - {sensor_in_stock}')
    
    
    # with open(f'data/{category_name}.csv', 'a', encoding = 'utf-8', newline='') as file:
        # writer = csv.writer(file)
        # writer.writerow((category_name, sensor_type, sensor_symbol, sensor_url, sensor_price))



Индуктивные датчики - Бесконтактный выключатель - ISB A11B-31N-1,5-L: https://teko-com.ru/product/isb-a11b-31n-1-5-l.html - 1 910 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISN ImP-12-16-LZ: https://teko-com.ru/product/isn-imp-12-16-lz.html - 2 680 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISN I8P-01G-35-L: https://teko-com.ru/product/isn-i8p-01g-35-l.html - 2 280 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISN FS2A-32N-6-L-C-O: https://teko-com.ru/product/isn-fs2a-32n-6-l-c-o.html - 4 850 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISN F4A-31N-8-LZ: https://teko-com.ru/product/isn-f4a-31n-8-lz.html - 1 620 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISN EC8A-31N-15-LZS4: https://teko-com.ru/product/isn-ec8a-31n-15-lzs4.html - 2 040 р. - 1-3 дня
Индуктивные датчики - Бесконтактный выключатель - ISN E71A-12G-14-LZ: https://teko-com.ru/product/isn-e71a-12g-14-lz.html - 3 260 р. - 1-3 дня
Инд

In [12]:
with open(f'data/{category_name}.csv', encoding = 'cp1251') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['sensor_full_name', 'sensor_url', 'sensor_price']
['Индуктивные датчики', 'Бесконтактный выключатель', 'ISB AF4B-31P-5-LZ-4,5', 'https://teko-com.ru/index.php?route=product/product&path=1437&product_id=17324', '1 350 р.']
['Индуктивные датчики', 'Бесконтактный выключатель', 'ISN ET8A-31N-15-LZ', 'https://teko-com.ru/product/isn-et8a-31n-15-lz.html', '1 990 р.']
['Индуктивные датчики', 'Бесконтактный выключатель', 'ISB A4B-31P-5-LZ-C', 'https://teko-com.ru/product/isb-a4b-31p-5-lz-c.html', '2 050 р.']
['Индуктивные датчики', 'Бесконтактный выключатель', 'ISB A4B-31P-5-LZ', 'https://teko-com.ru/product/isb-a4b-31p-5-lz.html', '1 600 р.']
['Индуктивные датчики', 'Бесконтактный выключатель', 'ISBt AC27B8-32P-4F-LTR9-C-P1-0,2', 'https://teko-com.ru/product/isbt-ac27b8-32p-4f-ltr9-c-p1-0-2.html', '3 340 р.']
['Индуктивные датчики', 'Бесконтактный выключатель', 'ISN F4A-31P-8-LZ', 'https://teko-com.ru/product/isn-f4a-31p-8-lz.html', '1 490 р.']
['Индуктивные датчики', 'Бесконтактный выключат

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 4328: character maps to <undefined>

In [ ]:
import re
s = sensor_list[0].find('img')["title"]
# print (sensor_list[0].find('img')["title"])
print (s)
s = re.sub("[а-яА-Я]+", '', s).strip()
print (s)
r = re.compile("[а-яА-Я]+")
words = ['apple', 'cat', 'яблоко', 'кот', 'building', 'машина', 'status',]
russian = [w for w in filter(r.match, words)]
print(russian)

Бесконтактный выключатель ISB A11B-31N-1,5-L-C
ISB A11B-31N-1,5-L-C
['яблоко', 'кот', 'машина']


In [53]:
import sqlite3

db = sqlite3.connect('dbtest.sqlite3')
cursor = db.cursor()
soup = BeautifulSoup(res.content, 'lxml')
sensor_list = soup.find_all('div', class_="caption")
id = int(2)
for sensor in sensor_list:
    sensor_bare_name = sensor.find('div', class_="tovarszg").text
    sensor_name = re.sub ( '\(.+\)', '', sensor_bare_name).strip()
    sensor_type = (re.sub("[a-zA-Z0-9-,]+", '', sensor_name).strip())
    sensor_type = ' '.join(filter(lambda x: len(x) > 3, sensor_type.split()))      #удалить короткие слова, фильтр русских букв в названии датчика.
    sensor_symbol = sensor_name.replace(sensor_type, '').strip()
    sensor_url = sensor.find('a').get("href")
    sensor_price = sensor.find('span', class_="category-price-test").text.strip()
    sensor_in_stock = sensor.find('p', class_="pricers").find('span').text.strip()
    # print (f'{category_name} - {sensor_type} - {sensor_symbol}: {sensor_url} - {sensor_price} - {sensor_in_stock}')

    
    test_url = str(id*10)
    row = (id, category_name, sensor_type, sensor_symbol, test_url, sensor_price, sensor_in_stock)
    cursor.execute("INSERT INTO main_sensors VALUES (?, ?, ?, ?, ?, ?, ?)", row)
    id = id+1
    # (id, category_name, sensor_type, sensor_symbol, sensor_url, sensor_price, sensor_in_stock)
    # cursor.execute('SELECT * FROM main_sensors')
    # print (cursor.fetchall())




db.commit()
db.close()